In [2]:
import zipfile
import librosa
import numpy as np
import librosa.display as dis
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.regularizers import l1, l2
from keras.layers import Dropout

### Preparación de data de entrenamiento

El objetivo de esta sección es el de leer los archivos de audio del dataset de entrenamiento y poder estardarizar todos los audios.

Esto se hace mediante dos funciones de la libreria librosa. La primera parte consiste en la transformacion del archivo a un arrelo unidimencional, de entre todos los arreglos se elije el de mayor tamaño para realizar una completacion de matrices, es decir, todos los audios son colocados dentro de arreglos unidimencionales deltamaño del audio de mayor variacion, estos arreglos estan compuestos originalmente de solo 0. Con esto rellenamos los audios con silencio hasta completar una duracion.

Luego estos arreglos de igual tamaño son transformados a matrices usando la funcion $librosa.feature.melspectrogram$, estas matrices se conocen como espectogramas y son una forma de visualizar en forma de imagen el audio.

In [3]:
dataset = pd.read_csv("Train.csv")

In [5]:
Xaux = []
Xlens = []
Xsr = []

for idFile in dataset["Id"].values:
    y, sr = librosa.load("Train/"+idFile, sr=22050, duration=15)
    Xaux.append(y)
    Xlens.append(len(y))
    Xsr.append(sr)

In [6]:
X = []
maxSize = np.max(Xlens)
for i, data in enumerate(Xaux):
    aux = np.zeros((1,maxSize))
    aux[0][:Xlens[i]] =  data
    melspect = librosa.feature.melspectrogram(y=aux, sr=sr, n_mels=128)
    X.append(melspect.T)
    
y = dataset["Expected"].values
labels_respuestas1 = []
labels_respuestas2 = []
for i in y:
    aux = i.split()
    labels_respuestas1.append( int(aux[0]) )
    labels_respuestas2.append( int(aux[1])-2 )
y1 = np.array( labels_respuestas1 )
y2 = np.array( labels_respuestas2 )

In [7]:
X_trS, X_testS, y_trS, y_testS = train_test_split(X, y1, test_size=0.3, train_size=0.7)
X_trS, X_valS, y_trS, y_valS = train_test_split(X_trS, y_trS, test_size=0.1, train_size=0.9)

X_trN, X_testN, y_trN, y_testN = train_test_split(X, y2, test_size=0.2, train_size=0.8)
X_trN, X_valtN, y_trN, y_valN = train_test_split(X_trN, y_trN, test_size=0.2, train_size=0.8)

### Modelos de predicción

Para clasificar los audios usaremos dos modelos, uno encargado de clasificar el sexo de la persona que habla en el audio y otro encargado de clasificar la nacionalidad del hablante.

El primer modelo, el encargado de clasificar el sexo, es un modelo generico con capas de convolucion. Se decidio utilizar capas de convolucion ya que con el tratamiento de datos estamos trabajando con "imagenes", por ello, se considero apropiado su uso. Al final para la capa de salida se decidio utilizar un solo nodo con una funcion de activacion sigmoidal ya que esta red solo clasifica si el hablante es hombre o mujer, es decir, una clasificacion binaria.

Para el segundo modelo, volvemos a utilizar capas de convolucion y con una capa de salida con 5 neuronas, las cuales representan las nacionalidades posibles, ya que estamos trabajando con un problema de multiples clases, se decidio cambiar la funcion sigmoidal por la funcion softmax.

In [8]:
modelSexo = models.Sequential()
modelSexo.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X[0].shape[0], 128, 1)))
modelSexo.add(layers.MaxPooling2D((2, 2)))

modelSexo.add(layers.Conv2D(64, (3, 3), activation='relu'))
modelSexo.add(layers.MaxPooling2D((2, 2)))

modelSexo.add(layers.Conv2D(64, (3, 3), activation='relu'))
modelSexo.add(layers.Flatten())

modelSexo.add(layers.Dense(1, activation='sigmoid'))
modelSexo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 535, 126, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 267, 63, 32)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 265, 61, 64)       18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 132, 30, 64)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 130, 28, 64)       36928     
                                                                 
 flatten (Flatten)           (None, 232960)            0

In [28]:
modelNa = models.Sequential()
modelNa.add(layers.Conv2D( 32, (3, 3), activation='relu', input_shape=(X[0].shape[0], 128, 1)) )
modelNa.add(layers.Conv2D( 32, (3, 3), activation='relu'))
modelNa.add(layers.MaxPooling2D((4, 4)))

modelNa.add(layers.Conv2D(64, (3, 3), activation='relu'))
modelNa.add(layers.Conv2D(64, (3, 3), activation='relu'))
modelNa.add(layers.MaxPooling2D((4, 4)))

modelNa.add(layers.Conv2D(64, (3, 3), activation='relu'))
modelNa.add(layers.Conv2D(64, (3, 3), activation='relu'))

modelNa.add(layers.Flatten())
modelNa.add(layers.Dense(32, activation='relu'))
modelNa.add(layers.Dense(64, activation='relu'))
modelNa.add(layers.Dense(5, activation='softmax'))

modelNa.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_59 (Conv2D)          (None, 535, 126, 32)      320       
                                                                 
 conv2d_60 (Conv2D)          (None, 533, 124, 32)      9248      
                                                                 
 max_pooling2d_26 (MaxPoolin  (None, 133, 31, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_61 (Conv2D)          (None, 131, 29, 64)       18496     
                                                                 
 conv2d_62 (Conv2D)          (None, 129, 27, 64)       36928     
                                                                 
 max_pooling2d_27 (MaxPoolin  (None, 32, 6, 64)        0         
 g2D)                                                

### Entrenamiento

##### Clasificacion sexo

In [24]:
modelSexo.compile(loss='binary_crossentropy',
              optimizer='adam', metrics='accuracy')

history1 = modelSexo.fit(np.array(X_trS), y_trS, epochs=5, 
                    validation_data=(np.array(X_valS), y_valS))

Epoch 1/5
28/28 [==============================] - 67s 2s/step - loss: 0.9292 - accuracy: 0.7107 - val_loss: 0.3161 - val_accuracy: 0.8673
Epoch 2/5
28/28 [==============================] - 63s 2s/step - loss: 0.1885 - accuracy: 0.9214 - val_loss: 0.0967 - val_accuracy: 0.9898
Epoch 3/5
28/28 [==============================] - 63s 2s/step - loss: 0.1248 - accuracy: 0.9567 - val_loss: 0.2288 - val_accuracy: 0.9388
Epoch 4/5
28/28 [==============================] - 63s 2s/step - loss: 0.0486 - accuracy: 0.9863 - val_loss: 0.0430 - val_accuracy: 0.9796
Epoch 5/5
28/28 [==============================] - 63s 2s/step - loss: 0.0372 - accuracy: 0.9875 - val_loss: 0.2159 - val_accuracy: 0.9694


##### Clasificacion nacionalidad

In [29]:
modelNa.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

history2 = modelNa.fit(np.array(X_trN), y_trN, epochs=30, 
    validation_data=(np.array(X_valtN), y_valN))

Epoch 1/30
28/28 [==============================] - 127s 4s/step - loss: 1.6408 - accuracy: 0.2096 - val_loss: 1.5971 - val_accuracy: 0.2321
Epoch 2/30
28/28 [==============================] - 118s 4s/step - loss: 1.5922 - accuracy: 0.2668 - val_loss: 1.5929 - val_accuracy: 0.2277
Epoch 3/30
28/28 [==============================] - 116s 4s/step - loss: 1.5622 - accuracy: 0.2814 - val_loss: 1.5757 - val_accuracy: 0.2411
Epoch 4/30
28/28 [==============================] - 118s 4s/step - loss: 1.5200 - accuracy: 0.3206 - val_loss: 1.6659 - val_accuracy: 0.2991
Epoch 5/30
28/28 [==============================] - 121s 4s/step - loss: 1.4519 - accuracy: 0.3688 - val_loss: 1.5423 - val_accuracy: 0.2857
Epoch 6/30
28/28 [==============================] - 117s 4s/step - loss: 1.3438 - accuracy: 0.4215 - val_loss: 1.7045 - val_accuracy: 0.2812
Epoch 7/30
28/28 [==============================] - 117s 4s/step - loss: 1.2284 - accuracy: 0.4832 - val_loss: 1.8351 - val_accuracy: 0.2991
Epoch 8/30
28

### Evaluacion

En esta sección aplicamos el mismo tratamiento que a los datos de entrenamiento.

In [30]:
datasetTest = pd.read_csv("Test.csv")

In [35]:
XauxTest = []
XlensTest = []
XsrTest = []
for idFile in datasetTest["Id"].values:
    y, sr = librosa.load("Test/"+idFile, sr=22050, duration=15)
    XauxTest.append(y)
    XlensTest.append(len(y))
    XsrTest.append(sr)

In [38]:
XTest = []
if np.max(XlensTest) > np.max(Xlens):
    maxSize = np.max(XlensTest)
else:
    maxSize = np.max(Xlens)
    
for i, data in enumerate(XauxTest):
    if i%100==0: print(i)
    aux = np.zeros((1,maxSize))
    aux[0][:XlensTest[i]] =  data
    melspect = librosa.feature.melspectrogram(y=aux, sr=sr, n_mels=128)
    XTest.append(melspect.T)

0
100
200
300
400
500


In [39]:
results1 = modelSexo.predict( np.array(XTest) )

19/19 [==============================] - 13s 665ms/step


In [40]:
results2 = modelNa.predict( np.array(XTest) )

19/19 [==============================] - 20s 1s/step


In [41]:
listPre = []
for i in range(len(results1)):
    auxindex = 0
    maxval = 0
    for k,j in enumerate(results2[i]):
        if j > maxval:
            maxval = j
            auxindex = k
    
    auxText = str( int(np.round(results1[i],1)) ) + " " + str( auxindex )
    listPre.append(auxText)

In [42]:
d = datasetTest["Id"].values
dt = pd.DataFrame( {"Expected":listPre}, index=d )

In [43]:
dt.to_csv('final.csv')

### Comentarios finales

El camino que se decidio tomar en este desafio fue trabajar con dos redes, una que prediga el sexo de la persona y otra que prediga la nacionalidad.
El rendimiento de la red de prediccion del sexo tiene una alta accuracy, en el caso de la red de nacionalidad no se tiene un buen rendimiento, se observa un overfitting de la red, lo cual hace que su prediccion sea pobre.